In [1]:
import open3d as o3d 
from visualization_utils import * 
import numpy as np 
from metrics_class import Metrics
from modified_LSTQ import evaluator

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
def intersect(pred_indices, gt_indices):
        intersection = np.intersect1d(pred_indices, gt_indices)
        return intersection.size / pred_indices.shape[0]

def iou(pred_indices, gt_indices):
        intersection = np.intersect1d(pred_indices, gt_indices)
        union = np.union1d(pred_indices, gt_indices)
        return intersection.size / union.size

def remove_semantics(labels,preds):
        gt_idcs = np.where(labels == 0)[0]
        new_ncuts_labels = preds.copy()
        for i in np.unique(preds):
                pred_idcs = np.where(preds == i)[0]
                cur_intersect = intersect(pred_idcs,gt_idcs)
                if cur_intersect > 0.8:
                        new_ncuts_labels[pred_idcs] = 0
        return new_ncuts_labels
        
def get_semantics(preds,gt_idcs,pcd):
        new_ncuts_labels = np.zeros_like(preds)
        new_pcd = o3d.geometry.PointCloud()
        new_pcd.points = pcd.points 
        num_clusters = 0 
        for i in np.unique(preds):
                #new_cols = np.zeros((np.asarray(pcd.points).shape[0],3))

                pred_idcs = np.where(preds == i)[0]
                cur_intersect = intersect(pred_idcs,gt_idcs)
                
                #new_cols[pred_idcs] = np.array([1,0,0])
                #new_pcd.colors = o3d.utility.Vector3dVector(new_cols)
                #o3d.visualization.draw_geometries([new_pcd])

                if cur_intersect > 0.1:
                        #print("intersect",cur_intersect)
                        new_ncuts_labels[pred_idcs] = 1
                        num_clusters += 1
        #o3d.visualization.draw_geometries([color_pcd_by_labels(pcd,new_ncuts_labels)])
        return np.where(new_ncuts_labels == 1)[0], num_clusters

def remove_semantics_pcd(pcd,labels,preds):
        cols = np.asarray(pcd.colors)
        gt_idcs = np.where(labels == 0)[0]
        new_ncuts_labels = preds.copy()
        for i in np.unique(preds):
                pred_idcs = np.where(preds == i)[0]
                cur_intersect = intersect(pred_idcs,gt_idcs)
                if cur_intersect > 0.8:
                        new_ncuts_labels[pred_idcs] = 0
                        cols[pred_idcs] = np.array([0,0,0])
                        
        pcd.colors = o3d.utility.Vector3dVector(cols)   
        return pcd

In [3]:
def remove_cols(pcd,cols,labels,unique_labels,idx=1): 

	new_pcd = o3d.geometry.PointCloud()
	new_pcd.points = pcd.points 
	new_cols = np.zeros((np.asarray(pcd.points).shape[0],3))
	pcd_cols = np.asarray(pcd.colors)
	
	idcs = np.where(labels == unique_labels[idx])[0]
	
	
	new_cols[idcs] = cols[idx]
	new_pcd.colors = o3d.utility.Vector3dVector(new_cols)
	#o3d.visualization.draw_geometries([new_pcd])
	return new_pcd,idcs 
	
	
def cluster_vis(pcd,cols,labels,unique_labels,idx=1): 

	new_pcd = o3d.geometry.PointCloud()
	new_pcd.points = pcd.points 
	new_cols = np.zeros((np.asarray(pcd.points).shape[0],3))
	pcd_cols = np.asarray(pcd.colors)
	
	idcs = np.where(labels == unique_labels[idx])[0]
	
	
	new_cols[idcs] = cols[idx]
	new_pcd.colors = o3d.utility.Vector3dVector(new_cols)
	#o3d.visualization.draw_geometries([new_pcd])
	return new_pcd,idcs 
	
	
	

In [4]:
out_folder = 'pcd_preprocessed/out_maps/'

fs = []
#fs.append('merge_a2_t3_0.05.pcd')
#fs.append('merge_a2_t3_0.025.pcd') 
#fs.append('merge_t3_0.05.pcd') 
#fs.append('ncuts_instances_tarl_spatial1.pcd') 
#fs.append('ncuts_sam1_instances.pcd')
#fs.append('ncuts_instances_tarl_spatial1.pcd')
#fs.append('hdbscan_instances.pcd') 
#fs.append('ncuts_a2_t3_0.025.pcd')
fs.append('ncuts_tarl_sam_spatial.pcd')
#fs.append('ncuts_sam.pcd')
#fs.append('ncuts_tarl_norm.pcd')

sem_gt = "merge_part_kitti_semantic7.pcd"
inst_gt = "merge_part_kitti_instance7.pcd"

pcd_gt = o3d.io.read_point_cloud(out_folder + inst_gt)
unique_colors, labels_kitti = np.unique(np.asarray(pcd_gt.colors),axis=0, return_inverse=True)

#o3d.visualization.draw_geometries([pcd_gt])
eval_lstq = evaluator()

for f in fs : 
	print("file name",f)
	pred_pcd = o3d.io.read_point_cloud(out_folder + f)
	#o3d.visualization.draw_geometries([pred_pcd])
	unique_colors, labels_ncuts = np.unique(np.asarray(pred_pcd.colors), axis=0, return_inverse=True)
	new_labels = remove_semantics(labels_kitti,labels_ncuts)
	metrics_pred = Metrics(name=f)
	metrics_pred.update_stats(new_labels,labels_kitti,calc_all=True,calc_lstq=True)
	
	#print("eval compute")
	#out = eval_lstq.add_batch(new_labels,labels_kitti)
	#print("AQ Overall",eval_lstq.get_eval()) 
	#eval_lstq.reset()
	
	#metrics_pred = Metrics(name=f)
	#metrics_pred.update_stats(new_labels,labels_kitti,calc_all=True)

file name ncuts_tarl_sam_spatial.pcd
Metrics for file ncuts_tarl_sam_spatial.pcd
{'panoptic': 0.7341087015052776, 'precision': 0.819047619047619, 'recall': 0.7962962962962963, 'fScore': 0.8075117370892019, 'usr': 0.05714285714285714, 'osr': 0.018518518518518517, 'noise': 0.13333333333333333, 'missed': 0.12037037037037036, 'mean': 0.9090997291896752}


AttributeError: 'Metrics' object has no attribute 'eval'

In [ ]:
'''
out_folder = 'pcd_preprocessed/'

fs = []
maps_folder = 'out_maps/'
#fs.append('merge_a2_t3_0.05.pcd')
#fs.append('merge_a2_t3_0.025.pcd') 
#fs.append('merge_t3_0.05.pcd') 
#fs.append('ncuts_instances_tarl_spatial1.pcd') 
#fs.append('hdbscan_instances.pcd') 


inst_gt = "merge_part_kitti_instance.pcd"

name = maps_folder + 'ncuts_tarl_spatial1.pcd'
#name = 'merge_t3_0.05.pcd'
sem_gt = 'merge_part_kitti_semantic7.pcd'

pcd_gt = o3d.io.read_point_cloud(out_folder + sem_gt)
pcd_pred = o3d.io.read_point_cloud(out_folder + name)

unique_colors_kitti, labels_kitti = np.unique(np.asarray(pcd_gt.colors),axis=0, return_inverse=True)
unique_colors, labels_pred = np.unique(np.asarray(pcd_pred.colors),axis=0, return_inverse=True)

o3d.visualization.draw_geometries([pcd_gt])
print(pcd_gt)
print(pcd_pred)

pcd_gt.remove_duplicated_points()
relevant_idcs = {1:'car',4:'fence',5:'trailer',6:'truck',7:'traffic-sign',10:'trunk',12:'pole',13:'other-object',15:'bike',16:'building',18:'pedestrian',19:'other structure'}
unique_labels = list(np.unique(labels_kitti))


for idx in relevant_idcs.keys(): 
	print(relevant_idcs[idx])
	#cluster_vis(pcd_gt,unique_colors,labels_pred,list(np.unique(labels_pred)),idx)
	gt_pcd, idcs = remove_cols(pcd_gt,unique_colors,labels_kitti,unique_labels,idx)
	#o3d.visualization.draw_geometries([gt_pcd])
	pred_labels, num_clusters = get_semantics(labels_pred,idcs,pcd_pred)
	cur_iou = iou(pred_labels,idcs)
	print("iou",cur_iou)
	print("num clusters",num_clusters)
	#pcd_cur = color_pcd_by_labels(pcd_pred,pred_labels,largest=False)
	#o3d.visualization.draw_geometries([pcd_cur])
	

'''

'\nout_folder = \'pcd_preprocessed/\'\n\nfs = []\nmaps_folder = \'out_maps/\'\n#fs.append(\'merge_a2_t3_0.05.pcd\')\n#fs.append(\'merge_a2_t3_0.025.pcd\') \n#fs.append(\'merge_t3_0.05.pcd\') \n#fs.append(\'ncuts_instances_tarl_spatial1.pcd\') \n#fs.append(\'hdbscan_instances.pcd\') \n\n\ninst_gt = "merge_part_kitti_instance.pcd"\n\nname = maps_folder + \'ncuts_tarl_spatial1.pcd\'\n#name = \'merge_t3_0.05.pcd\'\nsem_gt = \'merge_part_kitti_semantic7.pcd\'\n\npcd_gt = o3d.io.read_point_cloud(out_folder + sem_gt)\npcd_pred = o3d.io.read_point_cloud(out_folder + name)\n\nunique_colors_kitti, labels_kitti = np.unique(np.asarray(pcd_gt.colors),axis=0, return_inverse=True)\nunique_colors, labels_pred = np.unique(np.asarray(pcd_pred.colors),axis=0, return_inverse=True)\n\no3d.visualization.draw_geometries([pcd_gt])\nprint(pcd_gt)\nprint(pcd_pred)\n\npcd_gt.remove_duplicated_points()\nrelevant_idcs = {1:\'car\',4:\'fence\',5:\'trailer\',6:\'truck\',7:\'traffic-sign\',10:\'trunk\',12:\'pole\',13